## Preprocessing

In [27]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import shutil
from google.colab import files

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [28]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [29]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [30]:
# Binning APPLICATION_TYPE
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < 1000].index)
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

# Binning CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 1500].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')

In [31]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [32]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [33]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [34]:
# Define the filepath where you want to save the weights
checkpoint_path = "trained_charity_weights.{epoch:02d}.hdf5"

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,  # Save only the weights
    save_freq="epoch",  # Save weights every epoch
    period=5  # Save weights every five epochs
)
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer with Leaky ReLU activation and dropout
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
nn.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer with ELU activation
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='elu'))
nn.add(tf.keras.layers.BatchNormalization())

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 150)               6150      
                                                                 
 dropout_1 (Dropout)         (None, 150)               0         
                                                                 
 dense_5 (Dense)             (None, 60)                9060      
                                                                 
 batch_normalization_1 (Bat  (None, 60)                240       
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 30)                1830      
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                      

In [35]:
# Compiling the model with adjusted learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [36]:
# Train the model
# Train the model with the added callback
# Training the model with increased epochs
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs=250,
    callbacks=[checkpoint_callback]  # Add the ModelCheckpoint callback
)

Epoch 1/250
804/804 [==============================] - 4s 3ms/step - loss: 0.5925 - accuracy: 0.7045
Epoch 2/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5713 - accuracy: 0.7199
Epoch 3/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5674 - accuracy: 0.7243
Epoch 4/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5642 - accuracy: 0.7245
Epoch 5/250
794/804 [============================>.] - ETA: 0s - loss: 0.5627 - accuracy: 0.7269
Epoch 5: saving model to trained_charity_weights.05.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5627 - accuracy: 0.7269
Epoch 6/250
804/804 [==============================] - 3s 3ms/step - loss: 0.5609 - accuracy: 0.7273
Epoch 7/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5594 - accuracy: 0.7302
Epoch 8/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5588 - accuracy: 0.7292
Epoch 9/250
804/804 [=============================

In [37]:
# Evaluating the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5610 - accuracy: 0.7241 - 558ms/epoch - 2ms/step
Loss: 0.5610111951828003, Accuracy: 0.7240816354751587


In [38]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
# Download the file
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>